# CS 541-B Assignment 2 -- A * search and Uniform Cost Search on NYC Taxi dataset

#### Name: Enter your Name here
#### Stevens ID: Enter your Stevens ID here

## NYC Taxi Data
---
The task is to perform Uniform Cost Search and A∗ search. The dataset contains a list of the taxi trips made in NYC in the January 2015. 

Part 1. [20 pts] Represent the data as a graph. For the sake of simplicity, you can assume that edges are only between locations which had a valid taxi trip between them. 

Part 2. [20 pts] Implement the succAndCost fucntion that returns all the connected nodes from the passed node.
And implement the heuristic fucntion using geopy

#### Note: You can install geopy using the command
     pip install geopy

Part 3. [60 pts] 


*   Implement the Uniform Cost search where you can use the trip distances as the edge costs. The program should input two node ids from the user and output the path as well as the cost between them. 
*   Implement A∗ search using a heuristic. One idea of a heuristic value is to use straight line distance between 2 points. This can be computed using the geopy package. The program should input two node ids from the user and output the path as well as the cost between them. 

In [ ]:
# Required libraries
import pandas as pd
from tqdm import tqdm
from geopy import distance
import heapq, collections, re, sys, time, os, random

In [ ]:
"""
    Download the datset from the below link
"""
!wget 'https://raw.githubusercontent.com/GABudhrani/Assignment_2_NYC_TAxi/main/NYC_dataset.csv'

--2023-03-28 20:46:25--  https://raw.githubusercontent.com/GABudhrani/Assignment_2_NYC_TAxi/main/NYC_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8501184 (8.1M) [text/plain]
Saving to: ‘NYC_dataset.csv.2’

NYC_dataset.csv.2   100%[===================>]   8.11M  --.-KB/s    in 0.09s   

2023-03-28 20:46:25 (88.4 MB/s) - ‘NYC_dataset.csv.2’ saved [8501184/8501184]



In [ ]:
data_NYC = pd.read_csv("NYC_dataset.csv")
data_NYC.head(1)

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_distance,nodeid1,nodeid2
0,-73.9641,40.7614,-73.978,40.7831,2.4,48293,28440


In [ ]:
def plot_nodes_edges(node1, node2, cost, graph):
    """ Convert a graph in the format of pandas DF as Node 1, Node2, cost into dictionary. Replace the code between ##### with the correct implementation. 

        Note that your code needs to work for any input, not just for the one example below. 

        Input:
              Node 1, Node 2, Cost(	trip_distance) 
              
              example :- 48293, 28440, 2.40
        Output: 
              return graph (Add Key as Node and value as its relevent node and cost)

              Example :- 
              {48293: {2.4: 28440},
               28440: {2.4: 48293}}
              """

    node_cost = {}
    ##############################################START HERE##############################################
    # Part 1 (20 pt)
  
    graph = {1: {2.4: 2},
               2: {2.4: 1}}
    ##############################################END HERE################################################
    return graph
    

def create_nodes_edges(dataset):
    graph = {}
    for i in tqdm(range(dataset.shape[0])):
      graph = plot_nodes_edges(dataset.nodeid1[i], dataset.nodeid2[i], dataset.trip_distance[i], graph)
      graph = plot_nodes_edges(dataset.nodeid2[i], dataset.nodeid1[i], dataset.trip_distance[i], graph)
    return graph 

    
def extract_longitude_latitude(dataset):
    """ Input : 
              (nodeid1, pickup_latitude, pickup_longitude), Here we want to connect each node 
              with its pickup_longitude & pickup_latitude
              (nodeid2, pickup_latitude, pickup_longitude), Here we want to connect each node 
              with its dropoff_longitude & dropoff_latitude
        Output : 
              {node : [latitude, longitude]} """
    node_lat_long = {}
    for i in tqdm(range(dataset.shape[0])):
        node_lat_long[dataset.nodeid1[i]] = [dataset.pickup_latitude[i],dataset.pickup_longitude[i]]
        node_lat_long[dataset.nodeid2[i]] = [dataset.dropoff_latitude[i],dataset.dropoff_longitude[i]]
    return  node_lat_long

In [ ]:
graph = create_nodes_edges(data_NYC)

100%|██████████| 168152/168152 [00:12<00:00, 13207.55it/s]


In [ ]:
# calling create_nodes_edges fucntion
node_lat_long = extract_longitude_latitude(data_NYC)

100%|██████████| 168152/168152 [00:12<00:00, 12994.63it/s]


In [ ]:
# Data structure for supporting uniform cost search.
class PriorityQueue:
    def  __init__(self):
        self.DONE = -100000
        self.heap = []
        self.priorities = {}  # Map from state to priority

    # Insert |state| into the heap with priority |newPriority| if
    # |state| isn't in the heap or |newPriority| is smaller than the existing
    # priority.
    # Return whether the priority queue was updated.
    def update(self, node, newCost, newHistory):
        oldCost = self.priorities.get(node, None)
        if oldCost == None or newCost < oldCost:
            self.priorities[node] = newCost
            heapq.heappush(self.heap, (newCost, node, newHistory))
            return True
        return False

    # Returns (state with minimum priority, priority)
    # or (None, None) if the priority queue is empty.
    def removeMin(self):
        while len(self.heap) > 0:
            priority, state, history = heapq.heappop(self.heap)
            if self.priorities[state] == self.DONE: continue  # Outdated priority, skip
            self.priorities[state] = self.DONE
            return (state, priority, history)
        return (None, None, None) # Nothing left...


In [ ]:
class PathFinding(object):
    def __init__(self, graph, node_lat_long, start_position, end_position):
        self.graph = graph
        self.node_lat_long = node_lat_long
        self.start_node = start_position
        self.end_node = end_position
        
    def startState(self):
        return self.start_node
    
    def succAndCost(self, current_node):
        """ 
            In this Funciton we have to return all the nodes that are connected to the current_node. 
            
            Note that your code needs to work for any input, not just for the one example below. 

            Input:
                  curent_node
            output: 
            children: all the nodes that are connected to the current node and it's cost.
            Example: children = [(node1,cost1), (node2, cost2)]
        """
        # Generate children
        children = []
        ##############################################START HERE##############################################
        # Part 2.1 (10 pt)


        ##############################################END HERE################################################        
        return children
    
    def isEnd(self, current_node):
        return self.end_node == current_node
    
    def heuristic(self, node):
        """ 
          In this function you need to return the heuristic value of node. 
          Input: 
              Input will be the node
          Output: 
              Output will the distance between the node and the end node, please use distance.geodesic to compute the actual distance between two nodes
        """
        ##############################################START HERE##############################################
          # Part 2.2 (10 pt)

        return 0.0
        ##############################################END HERE################################################
        

In [ ]:
def printSolution(solution):
    totalCost, history = solution
    print_history = 'Starting Node' 
    for node in history:
        print_history += ' -> ' + str(node)
    print(print_history)
    print('totalCost: {}'.format(totalCost))

In [ ]:
def astar(problem):
    frontier = PriorityQueue()
    explored = set([])
    """ 
        In this function you have to write an algorithm to calculate the path with the minimum cost using A* Search 
        by calculating it's heuristic cost. Replace the code between the #### with the correct implementation. 
        Input:
              Input will be an object of the class PathFinding
        Output:
              return cost, history
              (history: Path from start node to the end node
              cost: Cost to travel from start node to end node)

                              OR

              return -1, []
              (if no path is found between Start node and End node)
    """

    ##############################################START HERE##############################################
    # Part 3 (30 pt)

    return 0, [1,2]
    ##############################################END HERE################################################

In [ ]:
def UCS(problem):
    """ 
        In this function you have to write an algorithm to calculate the path with the minimum cost using Uniform Cost Search and UCS 
        by calculating it's heuristic cost, cost respectively. Replace the code between the #### with the correct implementation. 
        Input:
              Input will be an object of the class PathFinding
        Output:
              return cost, history
              (history: Path from start node to the end node
              cost: Cost to travel from start node to end node)

                              OR

              return -1, [] 
              (if no path is found between Start node and End node)
    """
    frontier = PriorityQueue()
    explored = set([])
    ##############################################START HERE##############################################
    # Part 3 (30 pt)



    return 0, [1,2]
    ##############################################END HERE################################################

In [ ]:
"""
    Below are some of the test cases that you can try
    Start Node = 129891 | End Node = 7381
    Start Node = 51080 | End Node 79375

"""
start_node = 7381
end_node = 10666
problem = PathFinding(graph, node_lat_long, start_node, end_node) 

In [ ]:
# Astart Algorithm
# Calling the function
astart_path_cost = astar(problem)

# Checking if returned value have cost and history or not
if astart_path_cost[0] != -1:
    printSolution(astart_path_cost)

Starting Node -> 1 -> 2
totalCost: 0


In [ ]:
# UCS Algorithm
# Calling the function
astart_path_cost = UCS(problem)

# Checking if returned value have cost and history or not
if astart_path_cost[0] != -1:
    printSolution(astart_path_cost)

Starting Node -> 1 -> 2
totalCost: 0
